In [1]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy
from operator import itemgetter

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline(task = "fill-mask", model = 'bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

In [16]:
def masking(row):
    all_scores = []
    for i in range(len(row)): # каждое немаск предложение
        score = []
        sent = list(tokenizer(row[i]))
        len_sent = len(sent)
        for j in range(len_sent): # каждое маск предложение
    #         sent_id.append(i)
            sent_list = [str(word) for word in sent]
            masked = sent_list[j]
            sent_list[j] = "[MASK]"
            sent_list = " ".join(word for word in sent_list)
            for pred_token in classifier(sent_list):
                print(pred_token['token_str'], masked)
                if pred_token['token_str'] == masked:
                    score.append(pred_token['score'])
            print(score)
        all_scores.append(sum(score) / len_sent)
    return all_scores

df = pd.read_csv("realec - discourse.tsv", delimiter = "\t")

all_score_good = masking(df["texts_good"])
all_score_bad = masking(df["texts_bad"][:1000])

gold = []
result = []
for i in range(len(all_score_good[:1000])):
    gold.append(1)
    if all_score_good[i] > all_score_bad[i]:
        result.append(1)
    else:
        result.append(0)

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
print("Accuracy: %.4f" % accuracy_score(result, gold))

Accuracy: 0.6800
